# Set-up BW project

Import ecoinvent & biosphere databases and LCIs for LIB raw materials production

*Note: The results presented in the scientific article were produced with ecoinvent 3.10 cut-off system model*

In [1]:
%run _imports.ipynb

# Open BW project
BW_PROJECT = 'lib_rm' # insert your project name here
bw.projects.set_current(BW_PROJECT)

In [2]:
# Import ecoinvent database
EI_USERNAME = "xxxxx"           # add ecoinvent credentials
EI_PASS = "yyyyyyyyyyy"         # add ecoinvent credentials

if "ecoinvent-3.10-cutoff" in bw.databases:
    print('Ecoinvent database already installed')
else:
    bw.import_ecoinvent_release(
        version="3.10", 
        system_model='cutoff', 
        username=EI_USERNAME,
        password=EI_PASS
        )

Ecoinvent database already installed


In [3]:
# Import LIB raw materials LCIs
lci_lib_rms = bw.ExcelImporter(INVENTORIES_PATH)
lci_lib_rms.apply_strategies()
lci_lib_rms.match_database("ecoinvent-3.10-cutoff", fields=('name', 'reference product', 'unit', 'location'))
lci_lib_rms.match_database("ecoinvent-3.10-biosphere", fields=('name', 'unit', 'categories'))

Extracted 7 worksheets in 0.50 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 5.12 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields


In [4]:
# Add uncertainty data for pedigree matrix
for ds in lci_lib_rms.data:
    for exc in ds["exchanges"]:
        if "pedigree" in exc:

            # Pedigree are stored as strings
            pedigree_str = exc["pedigree"].strip("()")
            pedigre_scores = tuple([int(x) for x in pedigree_str.split(", ")])
            exc_amount = exc["amount"]

            uncertainty_dict = supporting_functions.create_pedigree_matrix(pedigre_scores, exc_amount)
            exc.update(uncertainty_dict)

In [5]:
# Write LCIs to database
lci_lib_rms.write_database()

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 08/29/2024 14:44:00
  Finished: 08/29/2024 14:44:00
  Total time elapsed: 00:00:00
  CPU %: 28.40
  Memory %: 2.32
Created database: LIB raw materials


In [6]:
for ds in lci_lib_rms.data:
    for exc in ds['exchanges']:
        if 'amount' not in exc:
            print("Missing `amount` in exc:")
            print("\t", exc)
            print("Dataset", ds['name'], ds['location'])
        elif 'input' not in exc:
            # check just to make sure
            print("Missing `input` in exc:")
            print("\t", exc)
            print("Dataset", ds['name'], ds['location'])  